In [1]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
from selenium.webdriver.common.by import By
import pandas as pd
from random import randint
from time import sleep
import re
import pickle

In [17]:
def scrape_30_listings(driver):
    # Loop through all 30 entries per page
    for post in range(29):
        
        # Pull job details from the list of jobs on left side of browser window
        listing_str = driver.find_element_by_xpath("(//li[@class='jl selected'])").text
        listing_int = listing_str.split("\n")
        
        # Clean up the data so order of elements is always ['Rating','Job_title','Company','Location', 'Salary_range', 'Status', 'Job_description', 'Company_info'] 
        if not bool(re.search(r'\d+.*', listing_int[0])): 
            listing_int.insert(0, "") # If no star rating available, insert a blank
        listing_lst = listing_int[0:2] + listing_int[2].split(" – ") + listing_int[3:] # Split company and location into 2 fields
        if 'EASY APPLY' in listing_lst: listing_lst.remove('EASY APPLY') # Remove "Easy Apply"
        if len(listing_lst) >= 5:
                if not bool(re.search(r'\D\d+[\w]\s*[-]\D\s*\d+\w+.*', listing_lst[4])): 
                    listing_lst.insert(4, "") # If no estimated salary available, insert a blank
        
        # Pull full text for job description from right side of browser window
        try:
            job_description = driver.find_element_by_xpath("(//div[@class='jobDescriptionContent desc'])").text
        except:
            job_description = '-'
        listing_lst.append(job_description)
        
        # Pull bulleted lists from job description from right side of browser window
        try:
            job_description_bullets = driver.find_element_by_xpath("(//div[@class='jobDescriptionContent desc']//ul[1])").text
        except:
            job_description_bullets = '-' 
        try:
            job_description_bullets = job_description_bullets + " " + driver.find_element_by_xpath("(//div[@class='jobDescriptionContent desc']//ul[2])").text
        except:
            pass  
        try:
            job_description_bullets = job_description_bullets + " " + driver.find_element_by_xpath("(//div[@class='jobDescriptionContent desc']//ul[3])").text
        except:
            pass
        try:
            job_description_bullets = job_description_bullets + " " + driver.find_element_by_xpath("(//div[@class='jobDescriptionContent desc']//ul[3])").text
        except:
            pass
        listing_lst.append(job_description_bullets)
        
        # If the company tab exists for this listing, click on it and scrape company info
        try:
            driver.find_element_by_xpath("(//li[@data-tab-type='overview'])").click()
            sleep(randint(1000,3000)/1000)
            company_str = driver.find_element_by_xpath("(//div[@id='EmpBasicInfo'])").text
        except:
            company_str = '-'
        listing_lst.append(company_str)
        
        # Get the link to the listing
        element = masked_driver.find_element_by_xpath("(//li[@class='jl selected']/div/a)")
        link = element.get_attribute('href')
        listing_lst.append(link)
        
        # Turn the data into a pandas Dataframe and add that df to the list, 'frames'
        listing_df = pd.DataFrame([listing_lst])
        frames.append(listing_df)
        
        # Click on next job posting from the list on left side of browser window
        driver.find_element_by_xpath("(//li[@class='jl'])[position()=" + str(post+1) + "]").click()
        sleep(randint(1000,3000)/1000)
        
    # Stack each entry into one large dataframe    
    features_df = pd.concat(frames)
    return features_df


In [18]:
def go_to_next_page(driver):
    driver.find_element_by_xpath("(//li[@class='next'])").click()
    sleep(randint(3000,5000)/1000)

In [19]:
def collect_mult_pages(masked_driver, start_page):
    
    scrape_30_listings(masked_driver)
    
    page_loop = 30-start_page
    for i in range(page_loop):
        go_to_next_page(masked_driver)
        sleep(randint(500,1000)/1000)
        x = scrape_30_listings(masked_driver)
        sleep(randint(500,1000)/1000)

    return x

In [5]:
# # Used to test proxy id's

# proxy = '79.134.214.186:32424'
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--proxy-server=%s' % proxy)
# masked_driver = webdriver.Chrome(executable_path="/Users/user/Documents/chromedriver2", options=chrome_options)
# masked_driver.get("https://whatismyipaddress.com/")

In [20]:
proxy_1 = '125.26.7.183:39018'
proxy_2 = '200.122.209.78:47220'
proxy_3 = '217.21.125.179:56774'
proxy_4 = '79.134.214.186:32424'

In [13]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % proxy_4)
masked_driver = webdriver.Chrome(executable_path="/Users/user/Documents/chromedriver2", options=chrome_options)
search_phrase='data+analyst'
masked_driver.get("https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=" + search_phrase + "&sc.keyword=" + search_phrase + "&locT=N&locId=1&fromAge=1&jobType=")


In [16]:
data_analyst = pd.concat(frames)
pickle.dump(data_analyst, open( "data_analyst_posts_pg_1-9.pkl", "wb" ) )

In [21]:
data_analyst = collect_mult_pages(masked_driver, 9)
pickle.dump(data_analyst, open( "data_analyst_posts.pkl", "wb" ) )
#output will have a few duplicates because had to rescrap half of pg 9

In [22]:
data_analyst

,0,1,2,3,4,5,6,7,8,9,10
0,3.8,"Data Analyst, Robotics",Amazon,"North Reading, MA",$42k-$61k(Glassdoor Est.),Today,Job Description\n\nAre you inspired by inventi...,-,"Overview\nHeadquartersSeattle, WA\nSize10000+ ...",https://www.glassdoor.com/partner/jobListing.h...,NaN
0,4.5,Data Analyst,Conductor,"New York, NY",$61k-$84k(Glassdoor Est.),Today,"At Conductor, a WeWork company, we're passiona...",3+ years of experience in a data analysis role...,"Overview\nHeadquartersNew York, NY\nSize201 to...",https://www.glassdoor.com/partner/jobListing.h...,NaN
0,3.8,Data Analyst,University of Texas Southwestern Medical Center,"Dallas, TX",$50k-$60k(Employer Est.),We're Hiring,Experience and education:\nEITHER\nMaster's de...,EITHER\nMaster's degree in field such as Biost...,"Overview\nHeadquartersDallas, TX\nSize10000+ E...",https://www.glassdoor.com/partner/jobListing.h...,NaN
0,4.2,Scientific Data Analyst and Programmer,Stanford University,"Stanford, CA",$63k-$92k(Glassdoor Est.),New,Scientific Data Analyst and Programmer\nStanfo...,Program and optimize pipelines for data qualit...,"Overview\nHeadquartersStanford, CA\nSize10000+...",https://www.glassdoor.com/partner/jobListing.h...,NaN
0,,Senior Data Analyst,qualifIQ,"New York, NY",$150k-$200k(Employer Est.),We're Hiring,"About Teachers Pay Teachers\nOur client, Teach...",Use data to tell a story. Surface insights and...,"Overview\nHeadquartersNew York, NY\nSize1 to 5...",https://www.glassdoor.com/partner/jobListing.h...,NaN
0,2.6,PANDAC Analyst,Owens & Minor,"Pittsburgh, PA",$35k-$56k(Glassdoor Est.),Today,Job Description\n\nJOB SUMMARY:\n\nProvides on...,Inside working conditions. No environmental ha...,"Overview\nHeadquartersMechanicsville, VA\nSize...",https://www.glassdoor.com/partner/jobListing.h...,NaN
0,3.9,Data Analyst - Analytic Solutions,Credit Acceptance Corporation,"Southfield, MI",$53k-$70k(Glassdoor Est.),We're Hiring,Our Analytics Department is on the frontline o...,Perform analysis on:\nPricing and credit risk\...,"Overview\nHeadquartersSouthfield, MI\nSize1001...",https://www.glassdoor.com/partner/jobListing.h...,NaN
0,4.0,Data Analyst / Database Developer,Advanced Digital Data,"Warwick, RI",,We're Hiring,ADD Systems’ industry-leading Business Intelli...,Document reporting requirements and create des...,"Overview\nHeadquartersFlanders, NJ\nSize51 to ...",https://www.glassdoor.com/partner/jobListing.h...,NaN
0,4.1,Data Analyst,The Motley Fool,"Alexandria, VA",$71k-$97k(Glassdoor Est.),New,Data Analyst\n\nAre you excited to use data to...,Consider yourself a scientist: Whether you hav...,"Overview\nHeadquartersAlexandria, VA\nSize201 ...",https://www.glassdoor.com/partner/jobListing.h...,NaN
0,3.9,Data Scientist,Mission Bell Manufacturing,"Morgan Hill, CA",,New,Mission Bell is the premiere destination for a...,Competitive pay\nRich benefits package\nA grow...,"Overview\nHeadquartersMorgan Hill, CA\nSize51 ...",https://www.glassdoor.com/partner/jobListing.h...,NaN
